In [30]:
import ccxt
import pandas as pd
import numpy as np

# Initialize Binance
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'  # hourly candles
limit = 1000  # number of candles to fetch

# Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)


In [ ]:
import ccxt
import pandas as pd
import numpy as np

# Initialize Binance
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'  # hourly candles
limit = 1000  # number of candles to fetch

# Fetch OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)

# Convert to DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)


In [ ]:
# ===============================
# 2. Calculate RSI
# ===============================
import ta
df["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()


In [33]:
import talib
df["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)

In [34]:
# ------------------------------
# 2. Calculate SMA (3 closes)
# ------------------------------
sma_window = 3
df['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)


In [35]:
# ------------------------------
# 2. Compute TSI
# ------------------------------
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi

df['TSI'] = compute_tsi(df['close'])


In [36]:
# ========================
# 2. Stochastic Oscillator
# ========================
period = 14
smooth_k = 3
smooth_d = 3

lowest_low = df["low"].rolling(period).min()
highest_high = df["high"].rolling(period).max()

df["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
df["%K"] = df["%K"].rolling(smooth_k).mean()
df["%D"] = df["%K"].rolling(smooth_d).mean()


In [37]:
# -------------------------
# 2) COMPUTE EXTRA INDICATORS (TA-Lib)
# -------------------------
# Using 'Close', 'High', 'Low', 'Volume' columns
close = df["close"].values.astype(np.float64)
high = df["high"].values.astype(np.float64)
low = df["low"].values.astype(np.float64)
volume = df["volume"].values.astype(np.float64)

In [38]:
# ATR
df["ATR"] = talib.ATR(high, low, close, timeperiod=14)

# Bollinger Bands width ( (upper - lower) / middle )
upper, middle, lower = talib.BBANDS(close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
df["BB_WIDTH"] = (upper - lower) / (middle + 1e-9)

# OBV
df["OBV"] = talib.OBV(close, volume)

# ADX (trend strength)
df["ADX"] = talib.ADX(high, low, close, timeperiod=14)

# MFI (volume-weighted oscillator)
df["MFI"] = talib.MFI(high, low, close, volume, timeperiod=14)

In [39]:
df.dropna(inplace=True)

In [40]:
df.head()

,open,high,low,close,volume,RSI,EMA12,EMA26,MACD,Signal,...,DEMA9,SMA,TSI,%K,%D,ATR,BB_WIDTH,OBV,ADX,MFI
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-08-19 20:00:00,113132.49,113570.44,112927.82,113570.44,496.52563,32.165705,114052.094249,114772.737472,-720.643223,-565.889173,...,113183.931296,113257.51,-36.979503,15.082655,11.533517,627.850339,0.031907,-2905.46565,51.428152,23.973847
2025-08-19 21:00:00,113570.44,113589.48,113413.14,113419.11,316.99786,30.822423,113954.712057,114672.468770,-717.756714,-596.262681,...,113165.545308,113374.01,-36.730084,19.523736,15.349185,595.599601,0.031902,-3222.46351,52.451644,27.622080
2025-08-19 22:00:00,113419.10,113440.43,113060.00,113174.42,1090.07931,28.732978,113834.667125,114561.502195,-726.835070,-622.377159,...,113075.832463,113387.99,-37.211789,19.973430,18.193274,580.230344,0.032014,-4312.54282,53.665900,22.129765
2025-08-19 23:00:00,113174.42,113201.26,112732.58,112872.94,721.99430,26.361790,113686.709106,114436.423514,-749.714408,-647.844609,...,112913.826944,113155.49,-38.400808,12.847809,17.448325,572.262462,0.033919,-5034.53712,55.005617,17.977261
2025-08-20 00:00:00,112872.95,113320.00,112843.14,113215.50,605.58560,33.115941,113614.215397,114345.984735,-731.769338,-664.629554,...,112931.829534,113087.62,-37.301604,10.977542,14.599594,565.448000,0.033958,-4428.95152,55.799013,20.299564


In [41]:
df.shape

(973, 21)

In [42]:
# ========================
# 3. Swing Highs / Lows
# ========================
lookback_swings = 5

def find_swings(prices, lb):
    labels = [0]*len(prices)
    for i in range(lb, len(prices)-lb):
        if prices[i] == max(prices[i-lb:i+lb+1]):
            labels[i] = -1   # swing high
        elif prices[i] == min(prices[i-lb:i+lb+1]):
            labels[i] = 1    # swing low
    return labels

df["Label"] = find_swings(df["close"].values, lookback_swings)


In [43]:
df.head()

,open,high,low,close,volume,RSI,EMA12,EMA26,MACD,Signal,...,SMA,TSI,%K,%D,ATR,BB_WIDTH,OBV,ADX,MFI,Label
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-08-19 20:00:00,113132.49,113570.44,112927.82,113570.44,496.52563,32.165705,114052.094249,114772.737472,-720.643223,-565.889173,...,113257.51,-36.979503,15.082655,11.533517,627.850339,0.031907,-2905.46565,51.428152,23.973847,0
2025-08-19 21:00:00,113570.44,113589.48,113413.14,113419.11,316.99786,30.822423,113954.712057,114672.468770,-717.756714,-596.262681,...,113374.01,-36.730084,19.523736,15.349185,595.599601,0.031902,-3222.46351,52.451644,27.622080,0
2025-08-19 22:00:00,113419.10,113440.43,113060.00,113174.42,1090.07931,28.732978,113834.667125,114561.502195,-726.835070,-622.377159,...,113387.99,-37.211789,19.973430,18.193274,580.230344,0.032014,-4312.54282,53.665900,22.129765,0
2025-08-19 23:00:00,113174.42,113201.26,112732.58,112872.94,721.99430,26.361790,113686.709106,114436.423514,-749.714408,-647.844609,...,113155.49,-38.400808,12.847809,17.448325,572.262462,0.033919,-5034.53712,55.005617,17.977261,0
2025-08-20 00:00:00,112872.95,113320.00,112843.14,113215.50,605.58560,33.115941,113614.215397,114345.984735,-731.769338,-664.629554,...,113087.62,-37.301604,10.977542,14.599594,565.448000,0.033958,-4428.95152,55.799013,20.299564,0


In [44]:
df['Label'] = df['Label'] + 1

In [45]:
df.head()

,open,high,low,close,volume,RSI,EMA12,EMA26,MACD,Signal,...,SMA,TSI,%K,%D,ATR,BB_WIDTH,OBV,ADX,MFI,Label
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-08-19 20:00:00,113132.49,113570.44,112927.82,113570.44,496.52563,32.165705,114052.094249,114772.737472,-720.643223,-565.889173,...,113257.51,-36.979503,15.082655,11.533517,627.850339,0.031907,-2905.46565,51.428152,23.973847,1
2025-08-19 21:00:00,113570.44,113589.48,113413.14,113419.11,316.99786,30.822423,113954.712057,114672.468770,-717.756714,-596.262681,...,113374.01,-36.730084,19.523736,15.349185,595.599601,0.031902,-3222.46351,52.451644,27.622080,1
2025-08-19 22:00:00,113419.10,113440.43,113060.00,113174.42,1090.07931,28.732978,113834.667125,114561.502195,-726.835070,-622.377159,...,113387.99,-37.211789,19.973430,18.193274,580.230344,0.032014,-4312.54282,53.665900,22.129765,1
2025-08-19 23:00:00,113174.42,113201.26,112732.58,112872.94,721.99430,26.361790,113686.709106,114436.423514,-749.714408,-647.844609,...,113155.49,-38.400808,12.847809,17.448325,572.262462,0.033919,-5034.53712,55.005617,17.977261,1
2025-08-20 00:00:00,112872.95,113320.00,112843.14,113215.50,605.58560,33.115941,113614.215397,114345.984735,-731.769338,-664.629554,...,113087.62,-37.301604,10.977542,14.599594,565.448000,0.033958,-4428.95152,55.799013,20.299564,1


In [46]:
import pandas as pd
import numpy as np
import talib
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from collections import Counter

FEATURES_1 = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

# We'll add these 5 extra indicators
EXTRA_FEATURES = ["ATR", "BB_WIDTH", "OBV", "ADX", "MFI"]

FEATURES = FEATURES_1 + EXTRA_FEATURES

X = df[FEATURES].values.astype(np.float32)
y = df["Label"].values.astype(np.int64)

# ===============================
# 2. Scale features
# ===============================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler.pkl")  # save scaler for later predictions


['scaler.pkl']

In [47]:
WINDOW_SIZE = 20               # longer sequence length (try 20-50)
BATCH_SIZE = 64
EPOCHS = 60
LR = 1e-3
HIDDEN_DIM = 128
NUM_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.3
OUTPUT_DIM = 3  

In [48]:
def create_sequences(X_array, y_array, window_size):
    X_seq, y_seq = [], []
    for i in range(window_size, len(X_array)):
        X_seq.append(X_array[i-window_size:i])   # shape: (window_size, features)
        y_seq.append(y_array[i])                 # label aligned to the last timestep
    X_seq = np.array(X_seq, dtype=np.float32)
    y_seq = np.array(y_seq, dtype=np.int64)
    return X_seq, y_seq

X_seq, y_seq = create_sequences(X_scaled, y, WINDOW_SIZE)
print("Sequence dataset shape:", X_seq.shape, y_seq.shape)  # (samples, window_size, features)


Sequence dataset shape: (953, 20, 16) (953,)


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)
print("Train shape:", X_train.shape, y_train.shape, "Test shape:", X_test.shape, y_test.shape)


Train shape: (762, 20, 16) (762,) Test shape: (191, 20, 16) (191,)


In [50]:
def oversample_by_duplication(X, y):
    counter = Counter(y)
    max_count = max(counter.values())
    indices_by_class = {cls: np.where(y == cls)[0] for cls in np.unique(y)}
    new_X = []
    new_y = []
    rng = np.random.default_rng(seed=42)
    for cls, idxs in indices_by_class.items():
        n_need = max_count - len(idxs)
        # include existing
        for i in idxs:
            new_X.append(X[i])
            new_y.append(y[i])
        # sample with replacement to balance
        if n_need > 0:
            choices = rng.choice(idxs, size=n_need, replace=True)
            for i in choices:
                new_X.append(X[i])
                new_y.append(y[i])
    # convert and shuffle
    new_X = np.array(new_X, dtype=np.float32)
    new_y = np.array(new_y, dtype=np.int64)
    # shuffle
    perm = rng.permutation(len(new_y))
    return new_X[perm], new_y[perm]

X_train_bal, y_train_bal = oversample_by_duplication(X_train, y_train)
print("After oversampling train shape:", X_train_bal.shape, y_train_bal.shape, "class counts:", Counter(y_train_bal))


After oversampling train shape: (1986, 20, 16) (1986,) class counts: Counter({np.int64(0): 662, np.int64(2): 662, np.int64(1): 662})


In [51]:
# -------------------------
# 10) Create PyTorch datasets/loaders
# -------------------------
train_tensor_X = torch.tensor(X_train_bal, dtype=torch.float32)
train_tensor_y = torch.tensor(y_train_bal, dtype=torch.long)
test_tensor_X = torch.tensor(X_test, dtype=torch.float32)
test_tensor_y = torch.tensor(y_test, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(train_tensor_X, train_tensor_y)
test_dataset = torch.utils.data.TensorDataset(test_tensor_X, test_tensor_y)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


In [52]:
# -------------------------
# 11) Define Bidirectional LSTM model
# Note: when bidirectional=True, hidden feature out size doubles
# -------------------------
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim,
                 bidirectional=True, dropout=0.0):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(input_dim,
                            hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers>1 else 0.0,
                            bidirectional=bidirectional)
        factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * factor, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # x shape: (batch, seq_len, features)
        out, (hn, cn) = self.lstm(x)          # out: (batch, seq_len, hidden*factor)
        out = out[:, -1, :]                   # take last time step
        out = self.dropout(out)
        out = self.fc(out)
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMModel(input_dim=X_seq.shape[2],
                    hidden_dim=HIDDEN_DIM,
                    num_layers=NUM_LAYERS,
                    output_dim=OUTPUT_DIM,
                    bidirectional=BIDIRECTIONAL,
                    dropout=DROPOUT).to(device)


In [53]:
# -------------------------
# 12) Loss, optimizer, class weights
# -------------------------
# Calculate class weights from original training distribution (before oversample) or from balanced set:
orig_counts = Counter(y_train)
total = sum(orig_counts.values())
# inverse frequency
weights = [total / orig_counts[c] if c in orig_counts else 1.0 for c in sorted(orig_counts.keys())]
# Ensure ordering for 0,1,2
# If any class missing in orig_counts, set weight 1.0
class_weights = torch.tensor([weights[0] if 0 in orig_counts else 1.0,
                              weights[1] if 1 in orig_counts else 1.0,
                              weights[2] if 2 in orig_counts else 1.0], dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5
)


In [54]:
# -------------------------
# 13) Training loop with validation
# -------------------------

SAVED_MODEL = "gregTech_4.pth"


def evaluate(model, loader, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            yb = yb.to(device)
            out = model(Xb)
            loss = criterion(out, yb)
            total_loss += loss.item() * Xb.size(0)
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += Xb.size(0)
    return total_loss / total, correct / total

best_val_loss = float("inf")
patience_counter = 0
PATIENCE = 40

for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    for Xb, yb in train_loader:
        Xb = Xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        preds = model(Xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * Xb.size(0)
    train_loss = epoch_loss / len(train_loader.dataset)
    val_loss, val_acc = evaluate(model, test_loader, device)
    scheduler.step(val_loss)
    print(f"Epoch {epoch}/{EPOCHS}  Train loss: {train_loss:.5f}  Val loss: {val_loss:.5f}  Val acc: {val_acc:.4f}")
    
    # early stopping
    if val_loss < best_val_loss - 1e-6:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), SAVED_MODEL)  # save best
        print("Saved best model.")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break

# load best model
model.load_state_dict(torch.load(SAVED_MODEL))
print("Training finished, best model loaded.")


Epoch 1/60  Train loss: 0.88228  Val loss: 1.55646  Val acc: 0.0471
Saved best model.
Epoch 2/60  Train loss: 0.83516  Val loss: 1.65004  Val acc: 0.0576
Epoch 3/60  Train loss: 0.75303  Val loss: 1.63350  Val acc: 0.0733
Epoch 4/60  Train loss: 0.53819  Val loss: 1.92846  Val acc: 0.0733
Epoch 5/60  Train loss: 0.39066  Val loss: 1.79478  Val acc: 0.0890
Epoch 6/60  Train loss: 0.34763  Val loss: 2.05965  Val acc: 0.0890
Epoch 7/60  Train loss: 0.27388  Val loss: 2.23299  Val acc: 0.0942
Epoch 8/60  Train loss: 0.23661  Val loss: 2.08976  Val acc: 0.0890
Epoch 9/60  Train loss: 0.20820  Val loss: 2.30628  Val acc: 0.1099
Epoch 10/60  Train loss: 0.19761  Val loss: 2.28514  Val acc: 0.1728
Epoch 11/60  Train loss: 0.21393  Val loss: 2.34009  Val acc: 0.0942
Epoch 12/60  Train loss: 0.19880  Val loss: 2.20987  Val acc: 0.1466
Epoch 13/60  Train loss: 0.19285  Val loss: 2.23987  Val acc: 0.1571
Epoch 14/60  Train loss: 0.18766  Val loss: 2.31492  Val acc: 0.1571
Epoch 15/60  Train loss: 